# Adversarial Training (MNIST)

In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

sys.path.append('../')
import torchattacks
from torchattacks import PGD, FGSM

from models import CNN

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load Data

In [3]:
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
batch_size = 128

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=batch_size,
                                         shuffle=False)

In [5]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/fashion_mnist_experiment_2')

## 2. Define Model

In [6]:
model = CNN().cuda()

In [7]:
dataiter = iter(train_loader)
images_, labels_ = dataiter.next()
writer.add_graph(model, images_.cuda())
writer.close()

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
atk = PGD(model, eps=0.3, alpha=0.1, steps=7)

## 3. Train Model

In [10]:
num_epochs = 5

In [11]:
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/5], lter [100/468], Loss: 2.3006
Epoch [1/5], lter [200/468], Loss: 1.9744
Epoch [1/5], lter [300/468], Loss: 1.9792
Epoch [1/5], lter [400/468], Loss: 1.6930
Epoch [2/5], lter [100/468], Loss: 1.4805
Epoch [2/5], lter [200/468], Loss: 1.0964
Epoch [2/5], lter [300/468], Loss: 1.2835
Epoch [2/5], lter [400/468], Loss: 0.9793
Epoch [3/5], lter [100/468], Loss: 0.9223
Epoch [3/5], lter [200/468], Loss: 0.7279
Epoch [3/5], lter [300/468], Loss: 0.8633
Epoch [3/5], lter [400/468], Loss: 0.6866
Epoch [4/5], lter [100/468], Loss: 0.7376
Epoch [4/5], lter [200/468], Loss: 0.5594
Epoch [4/5], lter [300/468], Loss: 0.7234
Epoch [4/5], lter [400/468], Loss: 0.5792
Epoch [5/5], lter [100/468], Loss: 0.6386
Epoch [5/5], lter [200/468], Loss: 0.5103
Epoch [5/5], lter [300/468], Loss: 0.6679
Epoch [5/5], lter [400/468], Loss: 0.5411


## 4. Test Model

### 4.1 Standard Accuracy

In [ ]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

### 4.2 Robust Accuracy

In [ ]:
model.eval()

correct = 0
total = 0

atk = FGSM(model, eps=0.3)

for images, labels in test_loader:
    
    images = atk(images, labels).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))